In [2]:
import pandas as pd
import re
import json
from konlpy.tag import Okt
okt = Okt()

df = pd.read_csv("data/중랑구최종데이터.csv")
df.head()

# 결측 처리(NaN 빈 문자열로 변경 + 전부 문자열로 통일 + 공백 제거)
df["name"] = df["name"].fillna("").astype(str).str.strip()
df["review_text"] = df["review_text"].fillna("").astype(str)

# 가게명 없는 건 카드 만들기 불가 → 제거
df = df[df["name"] != ""].copy()

# 공백 제거 후 글자수 계산
def review_len_no_space(text):
    return len(re.sub(r"\s+", "", text))

df["review_len"] = df["review_text"].apply(review_len_no_space)

# 전체 리뷰
before = len(df)
df = df[df["review_len"] > 20].copy()
# 20글자 초과 리뷰
after = len(df)

print("원본 리뷰 수(가게명 있는 것만):", before)
print("20글자 초과만 남김:", after, "삭제:", before-after)

# address2(지번) 없으면 "" 처리
if "address2" not in df.columns:
    df["address2"] = ""

df["address2"] = df["address2"].fillna("").astype(str).str.strip()

# 가게 구분 키(지점 섞임 방지: name + address2)
df["store_key"] = df["name"] + " | " + df["address2"]

# 가게별 리뷰 합치기
store_df = df.groupby("store_key").agg(
    name=("name", "first"),
    category=("category", "first"),
    address1=("address1", "first"),
    address2=("address2", "first"),
    review_count=("review_text", "count"),
    merged_review=("review_text", lambda x: "\n".join(["- " + str(t).strip() for t in x if str(t).strip()]))
).reset_index()

store_df.head()

# 청킹(문자 길이 기준)
def chunk_text(text, chunk_size=800, overlap=120):
    # text: 쪼갤 전체 문자열(가게 리뷰를 전부 합친 merged_review 같은 것)
    # chunk_size: 한 청크(조각)에 들어갈 최대 글자 수
    # overlap: 청크끼리 서로 겹치게(중복되게) 넣을 글자 수
    chunks = []
    start = 0
    while start < len(text):
        # 청크 크기 만큼 자르기 : end
        end = start + chunk_size
        # 슬라이싱 하여 chunks부분에 추가
        chunks.append(text[start:end])
        start = end - overlap  
        if start < 0:
            start = 0
    return chunks

# 단어 토큰화
token_pat = re.compile(r"[가-힣A-Za-z0-9]+")

# 불용어(원하면 계속 추가)
stopwords = set(["그리고","그러나","하지만","그래서","또한","그냥","너무","정말","진짜",
                 "조금","완전","매우","아주","있다","없다","하다","되다","같다","정도","때문"])

def get_top_keywords(text, top_k=20):
    # 형태소 분석 → 불필요 품사 제거 → 1글자 제거 → dict로 카운트

    # 1) 형태소 분석 + 원형화(stem=True)
    #    - Noun(명사), Adjective(형용사)만 남기면 조사/어미가 확 줄어듦
    tokens = [w for w, pos in okt.pos(text, stem=True) if pos in ("Noun", "Adjective")]

    # 2) 1글자 제거 + 숫자만 제거 + 불용어 제거
    tokens = [t for t in tokens if len(t) >= 2 and (not t.isdigit()) and (t not in stopwords)]

    if len(tokens) == 0:
        return []

    # 3) dict로 빈도 카운트 
    word_count = {}
    for token in tokens:
        word_count[token] = word_count.get(token, 0) + 1

    # 4) 상위 top_k 정렬해서 반환
    top_items = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return [[w, int(c)] for w, c in top_items]

out_path = "data/중랑구_store_cards(600,100).jsonl"

records = []

for i in range(len(store_df)):
    row = store_df.iloc[i]
    merged = row["merged_review"]

    # 청킹
    chunks = chunk_text(merged, chunk_size=600, overlap=100)

    # 전체 키워드 Top20
    top_keywords_store = get_top_keywords(merged, top_k=20)

    # 청크별 키워드 Top10
    chunk_list = []
    for idx, ch in enumerate(chunks):
        chunk_list.append({
            "chunk_id": idx,
            "text": ch,
            "top_keywords": get_top_keywords(ch, top_k=10)
        })

    rec = {
        "store_key": row["store_key"],
        "name": row["name"],
        "category": row["category"],
        "address1": row["address1"],
        "address2": row["address2"],
        "review_count": int(row["review_count"]),
        "top_keywords": top_keywords_store,
        "merged_review": merged,
        "chunks": chunk_list
    }

    records.append(rec)

# JSONL 저장
with open(out_path, "w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("저장 완료:", out_path)
print("가게 카드 수:", len(records))

원본 리뷰 수(가게명 있는 것만): 66399
20글자 초과만 남김: 37593 삭제: 28806
저장 완료: data/중랑구_store_cards(600,100).jsonl
가게 카드 수: 1558
